In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
import seaborn as sns
%matplotlib inline

In [ ]:
cpi_path = Path("CPI_data.csv")
cpi_df = pd.read_csv(cpi_path, index_col = 'DATE', parse_dates=True, infer_datetime_format=True)
cpi_df = cpi_df.sort_index()
cpi_df.head()

In [ ]:
housing_path = Path("Housing_data.csv")
housing_df = pd.read_csv(housing_path, index_col = 'DATE', parse_dates=True, infer_datetime_format=True)
housing_df = housing_df.sort_index()
housing_df.head()

In [ ]:
housing_df = housing_df.rename(columns={"MSPUS": "Medium_Housing_Prices"})
housing_df.head()

In [ ]:
combined_df = pd.concat([cpi_df,housing_df], axis='columns', join='inner')
combined_df